# Data Science Project

Este proyecto tiene como objetivo el poder obtener conocimiento a partir de los datos presentes en un conjunto de datos (dataset, por su traducción en ingles), lo que permitirá poder comprobar una o varias hipotesis propuestas.

Para esto, se define lo siguiente:

- **Conjunto de datos.** Se elegió el conjunto de datos "futbol" que contiene información relacionada de diversos jugadores de distintos clubes de todo el mundo; esto con su información estadistica como jugador y su algunos datos personales.


- **Hipotesis a comprobar.** ¿Existe alguna relación entre la edad, nacionalidad, club, pierna preferida, posición, fecha de unión al club, prestamo, fecha de contrado valido, altura, peso y clausula de liberación para poder determinar el valor y precio de venta de un jugador?

<img src="https://www.freewebheaders.com/wp-content/gallery/football/sports-soccer-stadium-night-scene-web-header.jpg" style="width: 100%"/>

## Importación de librerias

Se importando todas las librerias necesarias para este proyecto, donde las mas importante es **pandas** ya que permite la lectura, manipulación y almacenamiento de los datos.

In [ ]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn.preprocessing import MinMaxScaler
plt.style.use('ggplot')

## Lectura del conjunto de datos

Se lee el conjunto de datos elegido pero filtrando las columnas (variables, en Ciencia de Datos) unicamente necesarias, estas se describen a continuación.

- ID
- Age
- Nationality
- Club
- Value
- Wage
- Preferred Foot
- Position
- Joined
- Loaned From
- Contract Valid Until
- Height
- Weight
- Release Clause

Sin embargo, se renombran las columnas que poseen espacios con los mismos nombres pero sustituyendo los espacios con guiones bajos.

In [ ]:
dataframe = (pd.read_csv("futbol.csv")).filter(["ID", "Age", "Nationality","Club","Value", "Wage", "Preferred Foot", "Position", "Joined","Loaned From", "Contract Valid Until", "Height", "Weight", "Release Clause"]).rename(columns={"Preferred Foot":'Preferred_foot', 'Loaned From': 'Loaned_from', 'Contract Valid Until':'Contract_Valid_Until', 'Release Clause': 'Release_clause'})

Posteriormente, se visualiza el dataframe (estructura que posee un conjunto de datos) leido.

In [ ]:
dataframe

## Comprobación de los registros de datos

Debido a que el conjunto de datos elegido no fue elaborado por nosotros se procede a realizar una comprobación de los registros existentes en este, el cual tiene como objetivo de corroborar si existen registros nulos o vacios.
Comprobaremos el tipo de dato que tiene cada columna

In [ ]:
pd.DataFrame(dataframe.dtypes, columns=['Tipo de dato'])

Para disminuir el tiempo de procesamiento, cambiaremos el tipo de dato int64 a uno de menor cantidad de bits, pero primero debemos comprobar que ID y Age tengan el máximo aceptable por el tipo de dato.

In [ ]:
pd.DataFrame(dataframe.max(numeric_only=True), columns=['Tipo de dato'])

Primero comprobaremos que no haya ningún problema con este cambio

In [ ]:
(dataframe.ID.max() < np.iinfo(np.int32).max) & (dataframe.Age.max() < np.iinfo(np.int16).max)

In [ ]:
dataframe = dataframe.astype({"ID": np.int32, 'Age': np.int8})

Visualizar cantidad de datos nulos por columna

In [ ]:
pd.DataFrame(dataframe.isnull().sum(), columns=['Datos nulos'])

**Nota importante:** teniendo en cuenta el contexto de los datos, se puede mencionar que los datos nulos mas importantes son todos los que no pertenescan a **Release_clause** porque estos pueden categorizarse de otra manera.

## Modificación de los registros de datos

Como primer paso, primer visualizamos los registros que se intersectan con valores nulos o vacios de acuerdo a los siguientes datos:

- Preferred_foot
- Position
- Joined
- Loaned_from
- Contract_Valid_Until
- Height
- Weight

In [ ]:
dataframe[pd.isnull(dataframe[['Preferred_foot', 'Position', 'Joined', 'Loaned_from', 'Contract_Valid_Until','Height', 'Weight']]).all(axis=1)]

Posteriormente, se eliminan todos los registros que se intesectan partiendo de una columna común que en este caso es **Position**. Esto elimina un total de 48 registros del dataframe.

In [ ]:
dataframe.dropna(subset=['Position'], inplace=True)

Seguido a esto, se visualiza el dataframe.

In [ ]:
dataframe

Nuevamente, se procede a realizar una comprobación de los registros para corroborar en cuantos aun persisten registros nulos o vacios.

In [ ]:
pd.DataFrame(dataframe.isnull().sum(), columns=['Datos nulos'])

**Nota importante:** se puede observa que ahora existen 229 registros que no poseen un **Club** y **Contract_Valid_Until** por lo cual se puede deducir que hay una relación.

Partiendo de primer paso anterior, se procede a visualizar los registros que se intersectan con valores nulos o vacios de acuerdo a los siguientes datos:

- Joined
- Loaned_from
- Contract_Valid_Until

In [ ]:
Buscamos la intersección de las variables con valores nulos así como se hizo el paso anterior

In [ ]:
dataframe[pd.isnull(dataframe[['Joined', 'Loaned_from', 'Contract_Valid_Until']]).all(axis=1)]

Posteriormente, se eliminan todos los registros que se intesectan partiendo de una columna común que en este caso es **Contract_Valid_Until**. Esto elimina un total de 229 registros del dataframe.

In [ ]:
dataframe.dropna(subset=['Contract_Valid_Until'], inplace=True)

Seguido a esto, se visualiza el dataframe.

In [ ]:
dataframe

Nuevamente, se procede a realizar una comprobación de los registros para corroborar en cuantos aun persisten registros nulos o vacios.

In [ ]:
pd.DataFrame(dataframe.isnull().sum(), columns=['Datos nulos'])

**Nota importante:** se puede observar que la columna de **Joined** posiblemente sea excluyente con **Loaned From** ya que al hacer la suma aritmetica de sus valores nulos dan como resultado el total del dataframe.

Partiendo de lo observado, se procede a comprobar esta relación de excluyencia realizando la interseción de los registros de acuerdo a los siguientes criterios:

- Joined (con valores no nulos o vacios)
- Loaned_from

In [ ]:
dataframe[dataframe.Loaned_from.isnull() & dataframe.Joined.isnull()==False]

Con la operación anterior se puede comprobar de forma consistente la relación de excluyencia entre estas columnas, sin embargo; se realiza una ultima comprobación para validar si el tamaño de los registros intersectados es el mismo que el tamaño del mismo dataframe.

In [ ]:
dataframe[dataframe.Loaned_from.isnull() & dataframe.Joined.isnull()==False].values.shape == dataframe.shape

**Nota importante:** partiendo de la comprobación anterior, se puede verificar que el tamaño de los registros intersectados son iguales al tamaño del dataframe.

Una vez comprobado, validado y verificado la relación de excluyencia entre las columnas **Joined** y **Loaned_from**, es necesario realizar las modificaciones a los datos presentes en estes. Sin embargo, se necesita tener en cuenta lo siguiente:

- Las columnas son una cadena de texto con información compleja.
- La columna **Joined** se refiere a la fecha que el jugador se unió al club al que pertenece actualmente.
- La columna **Loaned_from** se refiere al club al que fue prestado incluyendo la fecha.

Las modificaciones propuestas consisten en realizar un renombrado de la columna **Joined** a **Loaned**, que sea de tipo booleana y que indiquen si un jugador está en préstamo (valor true indica que hay información en la columna **Loaned_from** y false indica que hay información en la columna **Joined**).

Como primer paso de las modificaciones propuestas, se renombra la columna **Joined** a **Loaned**.

In [ ]:
dataframe.rename({'Joined': 'Loaned'}, axis=1, inplace=True)

Posteriormente, se reemplazan los registros de la columna **Loaned** (anteriormente Joined) partiendo del siguiente criterio:
- Si presenta información entonces el nuevo valor del registro es true.
- Si no presenta información entonces el nuevo valor del registro es false.

In [ ]:
dataframe.Loaned = dataframe.Loaned.isnull()

Para comprobar que la operación haya provisto el remplazo esperado, se procede a visualizar la siguiente intersection de los registros de acuerdo a los siguientes datos:
- La columna **Loaned** en true
- La columna **Loaned_from** con registros nulos o vacios.

In [ ]:
dataframe[dataframe.Loaned == True].head(n=10)

Posteriormente, se visualiza la siguiente intersection de los registros de acuerdo a los siguientes datos:
- La columna **Loaned** en false
- La columna **Loaned_from** con registros nulos o vacios.

In [ ]:
dataframe[dataframe.Loaned == False].head(n=10)

Partiendo de lo anterior, se puede corroborar de forma consistente que la operación de reemplazamiento de los registros de la columna **Loaned** ha sido lo esperado.

Tomando lo mencionado, se procede a eliminar la columna **Loaned_from** debido a que su información se incluye en la columna **Loaned** de forma consistente.

In [ ]:
dataframe.drop(['Loaned_from'], axis=1, inplace=True)

Seguido a esto, se visualiza el dataframe.

In [ ]:
dataframe

Nuevamente, se procede a realizar una comprobación de los registros para corroborar en cuantos aun persisten registros nulos o vacios.

In [ ]:
pd.DataFrame(dataframe.isnull().sum(), columns=['Datos nulos'])

**Nota importante:** se puede observar que solamnente la columna **Release_clause** es la que contiene valores nulos o vacios.

Partiendo del contexto, los valores nulos o vacios existentes en los registros de la columna **Release_clause** puede inferir a que sea una cláusula de recesión de 0 euros, se procede a reemplazar los valores de los registros por una cadena "0".

In [ ]:
dataframe.loc[dataframe.Release_clause.isnull(),'Release_clause']='0'

Posteriormente, se comprueba que sean los mismos registros presentes en la columna **Release_clause** a los que se hayan reemplazado su valor nulo con la cadena "0".

In [ ]:
dataframe[dataframe.Release_clause =='0']

**Nota importante:** se puede observar que es la misma cantidad de los valores nulos o vacios iniciales de la columna **Release_clause** a los que se reemplazo con la cadena "0".

Nuevamente, se procede a realizar una comprobación de los registros para corroborar en cuantos aun persisten registros nulos o vacios.

In [ ]:
pd.DataFrame(dataframe.isnull().sum(), columns=['Datos nulos'])

**Nota importante:** se puede observar que ya no existen todos los valores nulos o vacios en todas las columnas del dataframe.

Una vez modificado los valores nulos o vacios, se procede a se analizar el tipo de dato de todas las columnas del dataframe.

In [ ]:
pd.DataFrame(dataframe.dtypes, columns=['Tipo de dato'])

**Nota importante:** se puede observar que todas las columnas corresponden a su tipo de dato con la excepción de las columnas **Value**, **Wage**, **Height** y **Release_clause** son tipo cadena (object), cuando realmente estos deberian ser de tipo numérico (int64).

Como primer paso, se procede a convertir las columnas **Value**, **Wage** y **Release_clause** que representan una cantidad de dinero evaluado en euros. Estos contienen de forma adicional lo siguiente:
- Signo de euro (€).
- Posfijo de miles (K) o millones (M).

Seguidamente, se comprueba que todos los registros posean un formato antes mencionado.

In [ ]:
all([re.search('€*(\d*.+\d*[MK]*|0)', value) for tupla in dataframe[['Value', 'Wage', 'Release_clause']] for value in tupla])

Una vez comprobado, se procede a eliminar los caracteres innecesarios y a convertir de forma inmediata los valores de miles o millones asi como tambien definir el nuevo valor de tipo flotante para los registros existentes de las columnas **Value**, **Wage** y **Release Clause**.

In [ ]:
dataframe[['Value','Wage', 'Release_clause']] = dataframe[['Value','Wage', 'Release_clause']].apply(lambda x: [tupla[1]*1000 if ('M' in tupla[0]) else tupla[1] for tupla in [(value, float(re.findall('(\d+(?:\.\d+)?)', value)[0])) for value in x]])

Como segundo paso, se procede a convertir las columna **Weight** que representan una cantidad de peso evaluado en libras. Estos contienen de forma adicional lo siguiente:
- Posfijo de libra (lbs).

Posteriormente, se comprueba que todos los registros posean un formato antes mencionado.

In [ ]:
all([re.findall('(\d+(?:\.\d+)?)lbs', value) for value in dataframe.Weight.values])

Una vez comprobado, se procede a eliminar los caracteres innecesarios y a definir el nuevo valor de tipo flotante de la columna **Weigth**.

In [ ]:
dataframe.Weight = dataframe.Weight.apply(lambda x: float(re.findall('(\d+(?:\.\d+)?)', x)[0]))

Como tercer paso, se procede a convertir las columna **Height** que representan una cantidad de altura evaluado en pulgadas. Estos contienen de forma adicional lo siguiente:
- Infijo de pulgadas (').

Seguidamente, se comprueba que todos los registros posean un formato antes mencionado.

In [ ]:
all([re.findall('(\d\'\d*)', value) for value in dataframe.Height])

Una vez comprobado, se procede a reemplazar el caracter de pulgada y a definir el nuevo valor de tipo flotante de la columna **Heigth**.

In [ ]:
dataframe.Height = [y[0]+(y[1] / 10) for y in [[int(y) for y in x.split("'")] for x in dataframe.Height]]

Como cuarto paso, se procede a convertir las columna **Contract_Valid_Until** que representan una fecha pero sin un formato establecido. Estos contienen de forma adicional lo siguiente:
- Mes (3 caracteres).
- Dia (1 a 2 caracteres).
- Año (4 caracteres).

Seguidamente, se comprueba que todos los registros posean por lo mínimo el año del contrato.

In [ ]:
all([re.findall('\d{4}', x)[0] for x in dataframe.Contract_Valid_Until])

Una vez comprobado, se procede a eliminar los caracteres innesesarios y a definir el nuevo valor de tipo entero de la columna **Contract_Valid_Until**.

In [ ]:
dataframe.Contract_Valid_Until = dataframe.Contract_Valid_Until.apply(lambda x: int(re.findall('\d{4}', x)[0]))

Por ultimo, se procede a comprobar los registros de la columna **Preferred_foot**.

In [ ]:
pd.DataFrame((lambda x:{'Preferred_foot':x.index})(dataframe.Preferred_foot.value_counts()))

**Nota importante:** se puede observar que esta columna solamente contiene valores entre las cadenas "Left" o "Righ".

Finalmente, se comprueba el tipo de dato de cada columna del dataframe.

In [ ]:
pd.DataFrame(dataframe.dtypes, columns=['Tipo de dato'])

Seguido a esto, se visualiza el dataframe.

In [ ]:
dataframe

**Nota importante:** se puede observar que las modificaciones han dado como resultado la reducción una columna y el completado de los datos en los valores de registros que antes contenian valores nulos o vacios.

## Normalización de los registros de datos

Partiendo de los datos modificados, se procede a crear una copia del dataframe con el objetivo de modificar los datos para posteriormente normalizarlos, pero sin modificar el dataframe original.

In [ ]:
normalizedDataframe = dataframe.copy()

Se visualiza la copia del dataframe creado.

In [ ]:
normalizedDataframe

**Nota importante:** debido a que existen columnas del dataframe de tipo cadena estos no se pueden normalizar, por lo que se requiere crear subconjunto de datos para relacionar estos valores para que posteriormente sean normalizados.

Como primer paso, se crea un subconjunto de datos para representar la frecuencia de los valores duplicados para la columna **Nationality**.

In [ ]:
countryDataframe = pd.DataFrame((lambda x:{'Nationality':x.index, 'Frequency':x.values})(dataframe.Nationality.value_counts()))

Se visualiza el nuevo subconjunto creado.

In [ ]:
countryDataframe

Como segundo paso, se crea un subconjunto de datos para representar la frecuencia de los valores duplicados para la columna **Club**.

In [ ]:
clubDataframe = pd.DataFrame((lambda x:{'Club':x.index, 'Frequency':x.values})(dataframe.Club.value_counts()))

Se visualiza el nuevo subconjunto creado.

In [ ]:
clubDataframe

Como tercer paso, se crea un subconjunto de datos para representar la frecuencia de los valores duplicados para la columna **Preferred_foot**.

In [ ]:
preferredFootDataframe = pd.DataFrame((lambda x:{'Preferred_foot':x.index, 'Frequency':x.values})(dataframe.Preferred_foot.value_counts()))

Se visualiza el nuevo subconjunto creado.

In [ ]:
preferredFootDataframe

Como cuarto paso, se crea un subconjunto de datos para representar la frecuencia de los valores duplicados para la columna **Position**.

In [ ]:
positionDataframe = pd.DataFrame((lambda x:{'Position':x.index, 'Frequency':x.values})(dataframe.Position.value_counts()))

Se visualiza el nuevo subconjunto creado.

In [ ]:
positionDataframe

Como quinto paso, se crea un subconjunto de datos para representar la frecuencia de los valores duplicados para la columna **Loaned**.

In [ ]:
loanedDataframe = pd.DataFrame((lambda x:{'Loaned':x.index, 'Frequency':x.values})(dataframe.Loaned.value_counts()))

Se visualiza el nuevo subconjunto creado.

In [ ]:
loanedDataframe

Como ultimo paso, se indexa las columnas **Nationality**, **Club**, **Preferred_foot**, **Position** y **Loaned** con los subconjuntos de datos creados en los pasos anteriores, esto sustituyendo los valores del nuevo dataframe con los nuevos pertenecienctes en los subconjuntos.

In [ ]:
for tupla in {'Nationality': countryDataframe, 'Club':clubDataframe, 'Preferred_foot':preferredFootDataframe, 'Position':positionDataframe, 'Loaned':loanedDataframe}.items(): normalizedDataframe[tupla[0]] = dataframe[tupla[0]].apply(lambda x:(tupla[1][tupla[0]].loc[tupla[1][tupla[0]] == x].index[0]))

Finalmente, se procede a normalizar todos los registros de todas las columnas del nuevo dataframe.

In [ ]:
normalizedDataframe = pd.DataFrame(MinMaxScaler().fit_transform(normalizedDataframe), columns=normalizedDataframe.columns)

Se procede a visualizar el nuevo dataframe con los registros normalizados.

In [ ]:
normalizedDataframe

## Visualización de los registros de datos

Partiendo del dataframe normalizados, se procede a generar gráficas con el objetivo de observar de forma visual el comportamiento de los datos.

Como primer paso, se visualiza las graficas de tipo histograma de todas las columnas del dataframe normalizado.

In [ ]:
normalizedDataframe.hist(figsize=(30,30))
plt.show()

Como segundo paso, se visualiza las gráficas de tipo dispersión todas las columnas a excepción de la columna **ID** común.

Se procede a visualizar gráfica de tipo dispersión entre la columna **ID** y **Age**.

In [ ]:
normalizedDataframe.plot.scatter(x = 'ID', y = 'Age', s = 10, c = 'red', figsize=(5,5));
plt.show()

Se procede a visualizar gráfica de tipo dispersión entre la columna **ID** y **Nationality**.

In [ ]:
normalizedDataframe.plot.scatter(x = 'ID', y = 'Nationality', s = 10, c = 'red', figsize=(5,5));
plt.show()

Se procede a visualizar gráfica de tipo dispersión entre la columna **ID** y **Club**.

In [ ]:
normalizedDataframe.plot.scatter(x = 'ID', y = 'Club', s = 10, c = 'red', figsize=(5,5));
plt.show()

Se procede a visualizar gráfica de tipo dispersión entre la columna **ID** y **Value**.

In [ ]:
normalizedDataframe.plot.scatter(x = 'ID', y = 'Value', s = 10, c = 'red', figsize=(5,5));
plt.show()

Se procede a visualizar gráfica de tipo dispersión entre la columna **ID** y **Wage**.

In [ ]:
normalizedDataframe.plot.scatter(x = 'ID', y = 'Wage', s = 10, c = 'red', figsize=(5,5));
plt.show()

Se procede a visualizar gráfica de tipo dispersión entre la columna **ID** y **Preferred_foot**.

In [ ]:
normalizedDataframe.plot.scatter(x = 'ID', y = 'Preferred_foot', s = 10, c = 'red', figsize=(5,5));
plt.show()

Se procede a visualizar gráfica de tipo dispersión entre la columna **ID** y **Position**.

In [ ]:
normalizedDataframe.plot.scatter(x = 'ID', y = 'Position', s = 10, c = 'red', figsize=(5,5));
plt.show()

Se procede a visualizar gráfica de tipo dispersión entre la columna **ID** y **Loaned**.

In [ ]:
normalizedDataframe.plot.scatter(x = 'ID', y = 'Loaned', s = 10, c = 'red', figsize=(5,5));
plt.show()

Se procede a visualizar gráfica de tipo dispersión entre la columna **ID** y **Contract_Valid_Util**.

In [ ]:
normalizedDataframe.plot.scatter(x = 'ID', y = 'Contract_Valid_Until', s = 10, c = 'red', figsize=(5,5));
plt.show()

Se procede a visualizar gráfica de tipo dispersión entre la columna **ID** y **Heigth**.

In [ ]:
normalizedDataframe.plot.scatter(x = 'ID', y = 'Height', s = 10, c = 'red', figsize=(5,5));
plt.show()

Se procede a visualizar gráfica de tipo dispersión entre la columna **ID** y **Weight**.

In [ ]:
normalizedDataframe.plot.scatter(x = 'ID', y = 'Weight', s = 10, c = 'red', figsize=(5,5));
plt.show()

Se procede a visualizar gráfica de tipo dispersión entre la columna **ID** y **Release_clause**.

In [ ]:
normalizedDataframe.plot.scatter(x = 'ID', y = 'Release_clause', s = 10, c = 'red', figsize=(5,5));
plt.show()

Como último paso, se visualiza la gráfica de tipo correlación para observar las relaciones mas significativas entre las columnas del dataframe normalizado.

In [ ]:
plt.figure(figsize=(30,30))
sb.heatmap(normalizedDataframe.corr(), cmap ='Blues', linewidths = 0.30, annot = True)
plt.show()